# REVIEWS SCRAPING

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException, ElementClickInterceptedException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time, requests
from bs4 import BeautifulSoup
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.options import Options
from time import sleep
import pandas as pd
import re
import string

# Scrape TrustPilot 

~23 pages. Need to scrape a page, and move to the next one. Repeat until reach last page.

In [17]:
def stringToDict(text):
    parts = text.split('","')
    vocab = {}
    
    for i in range(len(parts)):
        parts[i] = parts[i].replace('\n{"', '')
        parts[i] = parts[i].replace('}\n', '')
        pair = parts[i].split('":')
        pair[0] = pair[0].replace('"', '')
        pair[1] = pair[1].replace('"', '')
        vocab[pair[0]] = pair[1]
    return vocab

df = pd.DataFrame(columns=('name', 'date', 'stars', 'text', 'boaDate', 'boaText', 'source'))

In [67]:
driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())
url = 'https://www.trustpilot.com/review/www.bankofamerica.com'

temp = 2
driver.implicitly_wait(5)
driver.maximize_window()
driver.get(url)
xpath1 = '/html/body/div[4]/div[3]/div/div/div[3]/button'
xpath2 = '/html/body/div[3]/a'
# close accept cookies tab
driver.implicitly_wait(15)

# close Covid notification tab
try:
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, xpath2))).click()
except:
    pass

try:
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, xpath1))).click()
except TimeoutException:
    pass

driver.implicitly_wait(15)

# close Covid notification tab
try:
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, xpath2))).click()
except:
    pass
    

while True:   
    try:
        
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'lxml')

        for block in soup.find_all("article", class_='review'):
            # name, rating, review
            script = block.find("script").string
            vocab = stringToDict(script)
            name = vocab['consumerName']
            stars = int(vocab['stars'])
            text = vocab['reviewHeader']+vocab['reviewBody']

            #date
            dateBlock = block.find('div', class_='review-content-header__dates')
            dateScript = dateBlock.find("script").string
            date = dateScript.split('"')[3].split('T')[0]
            y, m, d = date.split('-')
            date = '/'.join([m, d, y])

            df = df.append({'name' : name , 'date' : date, 'stars': stars, 'text': text, 'boaDate': None, 'boaText': None, 'source': 'trustpilot.com'} , ignore_index=True)        
        
        next_page_btn = driver.find_element_by_partial_link_text("Next page")
        # open next page
        try:
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.PARTIAL_LINK_TEXT, 'Next page'))).click()
        except  StaleElementReferenceException:
            driver.implicitly_wait(5)
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.PARTIAL_LINK_TEXT, 'Next page'))).click()
        print('.........Opened page', temp)
        temp += 1
    except:
        print("No more pages left")
        break
print('processed all pages')        


[WDM] - Driver [C:\Users\mskac\.wdm\drivers\geckodriver\win64\v0.28.0\geckodriver.exe] found in cache


.........Opened page 2
.........Opened page 3
.........Opened page 4
.........Opened page 5
.........Opened page 6
.........Opened page 7
.........Opened page 8
.........Opened page 9
.........Opened page 10
.........Opened page 11
.........Opened page 12
.........Opened page 13
.........Opened page 14
.........Opened page 15
.........Opened page 16
.........Opened page 17
.........Opened page 18
.........Opened page 19
.........Opened page 20
.........Opened page 21
.........Opened page 22
.........Opened page 23
No more pages left
processed all pages


In [22]:
df.to_csv('data/trustpilot.csv')

# Scrape BBB.com

A single page, but needs 'load more' button to be pressed until scroll to the end

In [54]:
df = pd.DataFrame(columns=('name', 'date', 'stars', 'text', 'boaDate', 'boaText', 'source'))

In [55]:

# press load more until reach end of the page
driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())
url = 'https://www.bbb.org/us/nc/charlotte/profile/bank/bank-of-america-0473-100421/customer-reviews'
driver.implicitly_wait(5)
driver.maximize_window()
driver.get(url)

xpath = '/html/body/div[2]/div[2]/div/button'
css_selector = 'button.MuiButton-root:nth-child(2)'


try:
    # close cookies notification
    myElem = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, xpath)))
    myElem.click()
except TimeoutException:
    pass
driver.implicitly_wait(15)

while True:   
    try:   
        # load more reviews
        myElem = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, css_selector)))
        myElem.click()
        print('.........Loaded more reviews')
        sleep(5) # otherwise spinning circle blocks the button
    except TimeoutException:
        print("Done loading reviews")
        break    

[WDM] - Getting latest mozilla release info for v0.28.0
[WDM] - Trying to download new driver from https://github.com/mozilla/geckodriver/releases/download/v0.28.0/geckodriver-v0.28.0-win64.zip
[WDM] - Driver has been saved in cache [C:\Users\mskac\.wdm\drivers\geckodriver\win64\v0.28.0]


.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loaded more reviews
.........Loade

In [56]:
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'lxml')

In [58]:
reviews = []
for block in soup.find_all("div", class_='MuiGrid-root styles__Review-sc-1azxajg-0 fyMiFZ dtm-review MuiGrid-container'):
    stars = block.find_all(d='M259.3 17.8L194 150.2 47.9 171.5c-26.2 3.8-36.7 36.1-17.7 54.6l105.7 103-25 145.5c-4.5 26.3 23.2 46 46.4 33.7L288 439.6l130.7 68.7c23.2 12.2 50.9-7.4 46.4-33.7l-25-145.5 105.7-103c19-18.5 8.5-50.8-17.7-54.6L382 150.2 316.7 17.8c-11.7-23.6-45.6-23.9-57.4 0z')
    stars = len(stars)
    date = block.find("p", class_='MuiTypography-root Typography-y2r0fa-0 kpIiVF MuiTypography-body2').text
    name = block.find('p', class_='MuiTypography-root Name-t42m9k-0 kSwwPu MuiTypography-body2').text
    text = block.find_all('div', class_='MuiTypography-root Text-sc-12c66pm-0 fgbKlJ MuiTypography-body2')
    
    if len(text)>1:
        boaDate = block.find('p', class_='MuiTypography-root Date-sc-8slhbi-0 kEubpt MuiTypography-body1').text
        boaText = text[1].text
    else:
        boaDate = None
        boaText = None
        
    review = text[0].text
    
    df = df.append({'name' : name , 'date' : date, 'stars': stars, 'text': review, 'boaDate': boaDate, 'boaText': boaText, 'source': 'bbb.org'} , ignore_index=True)


In [59]:
df.head()

,name,date,stars,text,boaDate,boaText,source
0,scott o,11/21/2020,1,don't you set up an account with Bank of Ameri...,11/27/2020,Thank you for sharing your experience. At Bank...,bbb.org
1,Samy A,11/17/2020,1,Worst bank ever. I been a customer with Bank o...,None,None,bbb.org
2,Sharmell B,11/15/2020,1,Bank of America in McDonough ga 30253\n\nHas s...,None,None,bbb.org
3,Pam M,11/12/2020,1,They have absolutely HORRIBE CUSTOMER SERVICE!...,None,None,bbb.org
4,Gulshan G,11/04/2020,1,Have a checking account with direct deposit fo...,None,None,bbb.org


In [60]:
df.to_csv(r'data\bbb.csv')

# Scrape DepositAccounts.com

1. Click 'View more'
2. Find and click all 'Read More'
3. Result: single page with reviews, so just scrape it

In [104]:
driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())
url = 'https://www.depositaccounts.com/banks/bank-of-america.html'

driver.implicitly_wait(5)
driver.maximize_window()
driver.get(url)

[WDM] - Driver [C:\Users\mskac\.wdm\drivers\geckodriver\win64\v0.28.0\geckodriver.exe] found in cache


In [105]:
readMoreClassName = 'textExpand'

# click 'View more'
try:   
    
    viewMoreBtn = driver.find_element_by_partial_link_text("View MORE")
    viewMoreBtn.click()
except TimeoutException:
    print("Couldn't open all reviews page")

sleep(10)

# expand all reviews
more_buttons = driver.find_elements_by_class_name(readMoreClassName)
for x in range(len(more_buttons)):
    if more_buttons[x].is_displayed():
        driver.execute_script("arguments[0].click();", more_buttons[x])
        time.sleep(1)
        

In [124]:
df = pd.DataFrame(columns=('name', 'date', 'stars', 'text', 'boaDate', 'boaText', 'source'))

page_source = driver.page_source
soup = BeautifulSoup(page_source, 'lxml')
starClassRe = re.compile('^stars')

for block in soup.find_all("div", class_='bankReviewContainer'):
    title = block.find('h3').text
    
    stars = block.find('div', {"class" : starClassRe}).get('class')
    stars = int(stars[1][-1])
    
    user = block.find('span', itemprop='author').text
    date = block.find('span', itemprop='datePublished').get('datetime')
    y, m, d = date.split('-')
    date = '/'.join([m, d, y])
    
    text = block.find('p', itemprop='description').text
    review = title+' '+text
    
    df = df.append({'name' : user , 'date' : date, 'stars': stars, \
                    'text': review, 'boaDate': None, 'boaText': None, 'source': 'depositaccounts.com'} , ignore_index=True)

In [127]:
df.to_csv(r'data\depositaccounts.csv')

# Scrape ConsumerAffairs.com

1. Click read full review btn
2. Process data
3. Click Next btn
4. Repeat 1-3 until no Next btn

In [ ]:
driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())
url = 'https://www.consumeraffairs.com/finance/bofa.html'

driver.implicitly_wait(5)
driver.maximize_window()
driver.get(url)

In [ ]:
df = pd.DataFrame(columns=('name', 'date', 'stars', 'text', 'boaDate', 'boaText', 'source'))

monthVocab = {'jan': 1, 'feb': 2, 'march':3 , 'april':4, 'may': 5, 'june':6, 'july':7 , 'aug':8, 'sept':9 ,'oct':10, 'nov': 11,'dec':12}
page = 2

while True:   
    try:
        #expend all reviews
        more_buttons = driver.find_elements_by_partial_link_text('Read full review')
        
        for x in range(len(more_buttons)):
            if more_buttons[x].is_displayed():
                driver.execute_script("arguments[0].click();", more_buttons[x])
                time.sleep(1)
        # process data on current page        
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'lxml')

        for block in soup.find_all("div", class_='rvw js-rvw'):
            
            stars = block.find('meta', itemprop = 'ratingValue').get('content')
            stars = int(stars)
            
            user = block.find('strong', itemprop = 'author').text
            user = user.split('of')[0]
            
            date = block.find('span', class_ = 'ca-txt-cpt').text.split(': ')[1]
            date = date.replace('.', '').replace(',', '').lower()
            m, d, y = date.split(' ')
            m = str(monthVocab[m])
            date = '/'.join([m.zfill(2), d.zfill(2), y])
            
            text = block.find('div', class_ = 'rvw-bd').find_all('p')[1].text

            df = df.append({'name' : user.strip() , 'date' : date, 'stars': stars, 'text': text, 'boaDate': None, 'boaText': None, 'source': 'consumeraffairs.com'} , ignore_index=True)        
        
        next_page_btn = driver.find_element_by_partial_link_text("Next")
        # open next page
        try:
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.PARTIAL_LINK_TEXT, 'Next'))).click()
        except  StaleElementReferenceException:
            sleep(2)
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.PARTIAL_LINK_TEXT, 'Next'))).click()
        print('.........Opened next page', page)
        page += 1
    except:
        print("No more pages left")
        break
print('processed all pages')    

In [153]:
df.to_csv(r'data\consumeraffairs.csv')

# Scraping Wallethub.com

Using Scraping Robot saved locally all pages.
Scraping from local htmls.

In [35]:
from os import listdir
from os.path import isfile, join
import re

mypath = 'data\htmls'
files = [mypath+'\\'+ f for f in listdir(mypath) if isfile(join(mypath, f))]
print(len(files))
df = pd.DataFrame(columns=('name', 'date', 'stars', 'text', 'boaDate', 'boaText', 'product', 'source'))

244


In [36]:
for file in files:
    soup = BeautifulSoup(open(file,  encoding="utf8").read())

    for block in soup.find_all('article', itemprop = 'review'):
        user = block.find('span', itemprop='author').text
        date = block.find('time', class_='rvtab-ci-time regular-font').get('datetime')
        stars = block.find('meta', itemprop='ratingValue').get('content')
        text = block.find('div', itemprop='description').text
        product = block.find('div', class_='rvtab-ci-category')
        if product:
            product = product.text
            product = re.split('<.span>', product)
            product = product[0].split(': ')[1]
        else:
            product = None

        df = df.append({'name' : user.strip() , 'date' : date, 'stars': int(stars), 'text': text, \
                        'boaDate': None, 'boaText': None, 'product': product, 'source': 'wallethub.com'}\
                       , ignore_index=True) 

In [38]:
len(df)

2440

In [42]:
df.to_csv('data\wallethub.csv')